In [12]:
# Importing libraries
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape the data to have a channel dimension
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

# Convert the data to float32 and normalize it
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Convert the labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 65s 35ms/step - loss: 0.1188 - accuracy: 0.9642 - val_loss: 0.0551 - val_accuracy: 0.9826
Epoch 2/5
1875/1875 [==============================] - 66s 35ms/step - loss: 0.0441 - accuracy: 0.9862 - val_loss: 0.0433 - val_accuracy: 0.9865
Epoch 3/5
1875/1875 [==============================] - 66s 35ms/step - loss: 0.0273 - accuracy: 0.9911 - val_loss: 0.0375 - val_accuracy: 0.9884
Epoch 4/5
1875/1875 [==============================] - 65s 35ms/step - loss: 0.0174 - accuracy: 0.9943 - val_loss: 0.0448 - val_accuracy: 0.9878
Epoch 5/5
1875/1875 [==============================] - 64s 34ms/step - loss: 0.0119 - accuracy: 0.9960 - val_loss: 0.0437 - val_accuracy: 0.9880
